In [250]:
import random
import matplotlib.pyplot as plt
import numpy as np

num_villes=100
num_clients = num_villes - 1  # Exclure le dépôt

def generer_matrice(n):
    # Générer une matrice d'adjacence vide
    matrice = [[0] * n for _ in range(n)]

    # Générer des liens aléatoires
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < 0.6:  # Probabilité de lien entre deux villes
                weight = random.randint(1, 100)  # Poids du lien entre deux villes
                matrice[i][j] = weight 
                matrice[j][i] = weight
                

    return matrice

matrice = generer_matrice(num_villes)

KeyboardInterrupt: 

In [248]:
import random
import numpy as np
from numpy.random import choice as np_choice

# Évaluer la qualité des solutions
def calculer_distance_total(solution, matrice):
    distances = matrice[solution[:-1], solution[1:]]
    return np.sum(distances)

def maj_pheromone(matrice_pheromone, routes, distances):
    evaporation = 0.5

    # Évaporation des phéromones   
    matrice_pheromone *= (1 - evaporation)

    # Dépôt de phéromones sur les meilleures routes
    for route, distance in zip(routes, distances):
        pheromones = Q / distance
        matrice_pheromone[route[:-1], route[1:]] += pheromones

def maj_pheromone_intensification(matrice_pheromone, routes_visitees, intensite):
    for route in routes_visitees:
        for i in range(len(route) - 1):
            ville_actuelle = route[i]
            ville_suivante = route[i + 1]
            matrice_pheromone[ville_actuelle, ville_suivante] += intensite

def selectionner_ville(matrice_pheromone, current_ville, ville_non_visitee, alpha, beta):

    pheromone = matrice_pheromone[current_ville, ville_non_visitee]
    
    distance = matrice[current_ville, ville_non_visitee]

    proba = (pheromone ** alpha * (1.0 / distance) ** beta).astype(float)

    proba[distance == 0] = 0  # Gérer les distances nulles
    
    total_proba = np.sum(proba)
    if total_proba == 0:
        proba = np.ones(len(proba)) / len(proba)
    else:
        proba /= total_proba

    proba = np.maximum(proba, 0)

    next_ville_index = np_choice(len(ville_non_visitee), p=proba)
    next_ville = ville_non_visitee[next_ville_index]

    return next_ville

def vrp(matrice, num_fourmis, num_iterations, alpha, beta):
    num_ville = len(matrice)
    matrice_pheromone = np.ones((num_ville, num_ville))  # Matrice des phéromones
    best_route = None
    best_distance = float('inf')

    for i in range(num_iterations):
        routes_visitees = []  # Routes visitées par les fourmis
        routes = []
        distances = []

        for j in range(num_fourmis):
            route = [0]  # Dépôt initial
            ville_non_visitees = np.arange(1, num_ville)  # Villes non visitées
            while ville_non_visitees.size > 0:
                current_ville = route[-1]
                next_ville = selectionner_ville(matrice_pheromone, current_ville, ville_non_visitees, alpha, beta)
                route.append(next_ville)
                ville_non_visitees = np.delete(ville_non_visitees, np.where(ville_non_visitees == next_ville))

            route.append(0)  # Retour au dépôt

            if ville_non_visitees.size == 0 and route[-1] == 0:
                distance = calculer_distance_total(route, matrice)
                routes.append(route)
                distances.append(distance)

                if distance < best_distance:
                    best_route = route
                    best_distance = distance

                routes_visitees.append(route)

        maj_pheromone(matrice_pheromone, routes, distances)
        maj_pheromone_intensification(matrice_pheromone, routes_visitees, intensite)  # Intensification de la matrice de phéromone

    return best_route, best_distance

# On répartit entre le nombre de camions
def repartir_routes_optimal(route, num_camions):
    villes = route[1:-1]  # Liste des villes à répartir (excluant le dépôt)
    num_villes = len(villes)

    # Calcul de la distance totale de la route
    distance_totale = calculer_distance_total(route, matrice)

    # Calcul de la distance moyenne par camion
    distance_moyenne = distance_totale / num_camions

    camions_routes = []
    longueurs_routes = []

    camion_route = [0]  # Dépôt initial
    longueur_route = 0

    for i in range(num_villes):
        ville = villes[i]
        ville_suivante = villes[i + 1] if i < num_villes - 1 else 0  # Prochaine ville ou retour au dépôt

        # Ajout de la ville à la route actuelle
        camion_route.append(ville)
        longueur_route += matrice[ville][ville_suivante]

        # Si la longueur de la route actuelle dépasse la distance moyenne, on la termine et on commence une nouvelle route
        if longueur_route > distance_moyenne:
            camion_route.append(0)  # Retour au dépôt
            camions_routes.append(camion_route)
            longueurs_routes.append(longueur_route)

            camion_route = [0]  # Nouvelle route avec le dépôt initial
            longueur_route = 0

    # Ajout de la dernière route incomplète
    camion_route.append(0)  # Retour au dépôt
    camions_routes.append(camion_route)
    longueurs_routes.append(longueur_route)

    return camions_routes, longueurs_routes

# Paramètres de l'ACO
num_fourmis = 3
num_iterations = 15
alpha = 1.0  # Influence de la phéromone
beta = 5.0  # Influence de la visibilité   
Q = 100  # Quantité de phéromone déposée par les fourmis
intensite = 1.7 #intensification des pheromones
num_camions = 3  # Quantité de camions

best_route, best_distance = vrp(matrice, num_fourmis, num_iterations, alpha, beta)
camions_routes, longueurs_routes = repartir_routes_optimal(best_route, num_camions)

print("|-----------------Meilleur Global-----------------|")
print("Meilleure route:", best_route)
print("Distance totale:", best_distance)
print("|---------------Meilleur par camion---------------|")

# Affichage des routes assignées à chaque camion et de la longueur totale de chaque route
for i, route in enumerate(camions_routes):
    print(f"|------------------------{i+1}-----------------------|")
    print(f"Camion {i+1} - Route : {route} ")
    print(f"Distance totale : {longueurs_routes[i]}")
    

|-----------------Meilleur Global-----------------|
Meilleure route: [0, 957, 316, 255, 856, 887, 751, 79, 416, 431, 439, 894, 53, 355, 357, 52, 855, 358, 440, 380, 621, 879, 88, 6, 45, 27, 968, 694, 446, 484, 239, 642, 8, 221, 134, 510, 145, 469, 296, 115, 92, 785, 230, 298, 390, 851, 72, 244, 472, 801, 317, 638, 281, 565, 496, 984, 164, 180, 480, 242, 252, 753, 461, 877, 222, 746, 334, 1, 883, 434, 374, 535, 499, 182, 861, 605, 120, 610, 150, 966, 243, 216, 423, 823, 20, 315, 663, 294, 963, 24, 118, 259, 576, 32, 167, 202, 70, 437, 969, 506, 409, 871, 22, 14, 632, 688, 587, 414, 65, 405, 783, 267, 40, 585, 192, 858, 829, 741, 545, 939, 514, 460, 444, 918, 842, 567, 564, 498, 580, 73, 627, 965, 834, 377, 459, 607, 788, 467, 133, 247, 194, 755, 557, 975, 854, 618, 76, 151, 138, 807, 889, 609, 945, 513, 915, 639, 422, 927, 733, 990, 874, 799, 144, 508, 520, 555, 705, 392, 953, 901, 78, 869, 378, 147, 384, 176, 550, 33, 730, 261, 669, 962, 862, 154, 653, 791, 707, 748, 313, 292, 100, 353

In [238]:
import networkx as nx
import matplotlib.pyplot as plt

import networkx as nx
import matplotlib.pyplot as plt

def plot_weighted_matrix(matrix, paths):
    
    matrix = [[-1 if val == 0 else val for val in row] for row in matrix]
    # Créer un graphe orienté
    G = nx.DiGraph()

    # Ajouter les nœuds
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            G.add_node((i, j))

    # Ajouter les arêtes avec les poids correspondants
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            for k, l in [(i+1, j), (i, j+1)]:
                if k < len(matrix) and l < len(matrix[0]):
                    G.add_edge((i, j), (k, l), weight=matrix[i][j])
    
    # Créer un dictionnaire de positions pour les nœuds
    pos = {(i, j): (j, -i) for i in range(len(matrix)) for j in range(len(matrix[0]))}

    # Créer une liste de couleurs pour les arêtes selon les chemins
    edge_colors = ['r', 'g', 'b']

    # Créer une liste de styles de flèches pour les arêtes
    arrow_styles = ['-', '->', '-|>']

    # Afficher le graphe avec les arêtes et les nœuds
    plt.figure(figsize=(8, 6))
    for i, path in enumerate(paths):
        edges = [(tuple(path[n]), tuple(path[n+1])) for n in range(len(path)-1)]
        if edges:
            nx.draw_networkx_edges(G, pos, edgelist=edges,
                                edge_color=edge_colors[i % len(edge_colors)], arrowsize=10,
                                connectionstyle=f'arc3,rad={0.1*i}', arrowstyle=arrow_styles[i % len(arrow_styles)])

    nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=500)
    nx.draw_networkx_labels(G, pos, font_color='black')
    
    # Afficher les poids des arêtes
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')

    # Masquer les axes
    plt.axis('off')

    # Afficher le graphique
    plt.show()


plot_weighted_matrix(matrice, camions_routes)


TypeError: 'int' object is not iterable

<Figure size 800x600 with 0 Axes>